In [ ]:
import numpy as np
from numpy import random
from numpy import math
import matplotlib  
import matplotlib.pyplot as plt 

# copy
import copy


# import Keras and TF
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.models import clone_model
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Add, Dense, Activation, Flatten, Conv2D, Conv1D, MaxPooling2D, Dropout,BatchNormalization, Input, concatenate, Lambda
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


from numpy import linalg as LA
from tensorflow.python.keras import backend as K

In [ ]:
# function for calculating the winding number
def winding(state, N_k): 

    # phases
    Theta = np.zeros(N_k)
    for i_x in range(N_k) : 
        
        z = state[i_x, 0] + 1j * state[i_x, 1]
        Theta[i_x] = np.angle(z)  # -pi to pi
    
    # winding
    w = 0            
    for i_x in range(N_k) :
        
        delta_Theta = Theta[i_x] - Theta[i_x-1] #-2 *pi to 2*pi
                
        w += (delta_Theta + math.pi) % (2 * math.pi) - math.pi          
        
    return w / (2 * math.pi)


# Pauli matrices

sigmax = np.array([[0, 1], [1, 0]], dtype=complex)
sigmay = np.array([[0, -1j], [1j, 0]], dtype=complex)


# extract Pauli components
def ToSigmaZ(H):
    
    f = np.real((H[0,0] - H[1,1])/2)
    
    return f

def ToSigmaX(H):
    
    f = (np.real(H[0,1]) + np.real(H[1,0]))/2
    
    return f

def ToSigmaY(H):
    
    f = (np.imag(H[0,1]) - np.imag(H[1,0]))/2
    
    return f

# construct states: trivial, omega = 0, omega = 1 and omega = random
def AIII_state_trivial(N_k):
        
    state = np.zeros((N_k, 2), dtype = float)
        
    for i_x in range(N_k) : 
            
        state[i_x, 0] = 1 
        state[i_x, 1] = 0 
            
    return state 

def AIII_state_w_0(N_k):
    
    state = np.zeros((N_k, 2), dtype = float)

    # k in (0, pi)
    for i_x in range(round(N_k/2)) : 
            
        k_x = 2 * math.pi * i_x/ (N_k)
                
        h_x = math.cos(k_x)
        h_y = math.sin(k_x)

        E = (h_x**2 + h_y**2)**0.5
    
        state[i_x, 0] = h_x/E 
        state[i_x, 1] = h_y/E    
        
    # k in (pi, 2*pi)   
    for i_x in range(round(N_k/2)) : 
            
        k_x = 2 * math.pi * i_x/ (N_k)
                
        h_x = math.cos(-k_x)
        h_y = math.sin(-k_x)

        E = (h_x**2 + h_y**2)**0.5
    
        state[i_x + round(N_k/2), 0] = h_x/E 
        state[i_x + round(N_k/2), 1] = h_y/E    
        
        
    return state 

def AIII_state_w_1(N_k):
    
    state = np.zeros((N_k, 2), dtype = float)

    for i_x in range(N_k) : 
            
        k_x = 2 * math.pi * i_x/ (N_k)
                
        h_x = math.cos(k_x)
        h_y = math.sin(k_x)

        E = (h_x**2 + h_y**2)**0.5
    
        state[i_x, 0] = h_x/E 
        state[i_x, 1] = h_y/E    
        
    return state     

def AIII_state_w_random(N_k):
    
    state = np.zeros((N_k, 2), dtype = float)

    for i_x in range(N_k) : 
            
                
        h_x = random.random() - 0.5
        h_y = random.random() - 0.5

        E = (h_x**2 + h_y**2)**0.5
        
        state[i_x, 0] = h_x/E 
        state[i_x, 1] = h_y/E 
        
        
    return state 

# perform a random deformation
def make_move(state, N_k):
    
    # look for a random deformation that fulfills the continuity criterion
    condition = True
    
    while condition:
        
        # pick a site and an angle
        index = random.randint(0, N_k - 1)
        phase = 2*math.pi * (random.random() - 0.5)
     

        # check if the rotation for phase is continuous:
        z = state[index, 0] + 1j * state[index, 1] # chosen state
           
        # state on the left side
        if index > 0 :
            z_left = state[index - 1, 0] + 1j * state[index - 1, 1]
        else :
            z_left = state[N_k - 1, 0] + 1j * state[N_k - 1, 1]
        
        # state on the right side
        if index < (N_k - 1) :
            z_right = state[index + 1, 0] + 1j * state[index + 1, 1]
        else :
            z_right = state[0, 0] + 1j * state[0, 1]
                
        # current phase differences      
        phase_diff1 = np.angle(z) - np.angle(z_left)
        phase_diff2 = np.angle(z) - np.angle(z_right) 
            
        if phase_diff1 > math.pi :
            phase_diff1 -= 2*math.pi
            
        if phase_diff1 < -math.pi :
            phase_diff1 += 2*math.pi
            
        if phase_diff2 > math.pi :
            phase_diff2 -= 2*math.pi
            
        if phase_diff2 < -math.pi :
            phase_diff2 += 2*math.pi
        
        # check that the phase differences won't violate the continuity criterion after the random deformation:
        if (np.abs(phase_diff1 + phase) < math.pi) and np.abs(phase_diff2 + phase) < math.pi:
            # if satisfied -> exit the loop 
            condition = False
            
    
    # rotate the state by the chosen random angle
    z = state[index, 0] + 1j * state[index, 1]
    z *= np.exp(1j * phase)
            
    state[index, 0] = np.real(z)
    state[index, 1] = np.imag(z)
      
    return state


# create the datasets for the training:
def create_data(state, N_samples, N_k, N_moves): 
    
    # winding (just to verify)
    print('-----------------------------------')
    print(winding(state, N_k))
    
    # Generate random data
    generated_states = np.zeros((N_samples, N_k + 1, 2), dtype = 'float')
    for step in range(N_samples):
    
        state_new = copy.deepcopy(state)
        
        for move_per_step in range(N_moves) :
                
            # make a move
            state_new = make_move(state_new, N_k)
            
          
        # winding (just to verify)
        print(winding(state_new, N_k))
        
        # save
        generated_states[step, :N_k, :] = state_new
        
        # impose periodic boundary conditions
        generated_states[step, N_k, :] = generated_states[step, 0, :]        
        
        print(step)
        
    return generated_states
    


In [ ]:
# produce the datasets
N_moves = 1000
N_k = 100
N_sample = 10000


#--------------------------------------------
# trivial
state = AIII_state_trivial(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves)

Path = 'data_AIII\\trivial.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case w = 0
state = AIII_state_w_0(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves)

Path = 'data_AIII\\w_0.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case w = 1
state = AIII_state_w_1(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves)

Path = 'data_AIII\\w_1.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case w = random
state = AIII_state_w_random(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves)

Path = 'data_AIII\\w_random.npy'
np.save(Path,  generated_states)


In [ ]:
# interpolate from N_k + 1 to 2*N_k + 1 k-points
def expend(data):
    
    N_k = data.shape[1] - 1 
    data_new = np.zeros((data.shape[0], 2*N_k+1, 2), dtype = float)
    
    for n in range(data.shape[0]):    
        
        for i_x in range(N_k + 1):
            data_new[n, 2*i_x, :] = data[n, i_x, :]
    
        for i_x in range(N_k): 
            data_new[n, 2*i_x + 1, :] = interpolate(data_new[n, 2*i_x, :], data_new[n, 2*i_x + 2, :])
       
  
    return data_new


# interpolation between two vectors          
def interpolate(H1, H2):
       
    H_int = (H1 + H2)/np.linalg.norm(H1 + H2)
    
    return H_int
    


In [ ]:
# do the interpolation and save the data

Path = 'data_AIII\\trivial.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_AIII_expended\\trivial.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_AIII\\w_0.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_AIII_expended\\w_0.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_AIII\\w_1.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_AIII_expended\\w_1.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_AIII\\w_random.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_AIII_expended\\w_random.npy'
np.save(Path,  expended_data)